In [ ]:
! pip install tensorflow-addons
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.callbacks import ReduceLROnPlateau, Callback, ModelCheckpoint
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.utils import to_categorical
from keras import backend as K
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm import tqdm
from sklearn.utils import class_weight


import keras
from keras import Input
from keras.models import Model
from keras.utils import *
from keras.layers import *


import matplotlib.pyplot as plt

import gc

In [ ]:
'''
    Config
'''
IMG_SIZE = 224
BATCH_SIZE = 16

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): 
            return img 
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img
    
def load_ben_color(image, sigmaX=10):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image).astype('uint8')
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

'''
    Preprocessing for ImageDataGenerator since ImageDataGenerator reads images in rgb mode, while opencv in bgr
'''
def preprocessing(image, sigmaX=10):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = crop_image_from_gray(image).astype('uint8')
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image.astype('float32') / 255.

## load train and test image 

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,weights=None,input_shape=(IMG_SIZE,IMG_SIZE,3))

In [ ]:
flatten_layer = tf.keras.layers.Flatten()
dense_layer_1 = tf.keras.layers.Dense(4096, activation='relu')
Dropout_1 = tf.keras.layers.Dropout(0.6)
dense_layer_2 = tf.keras.layers.Dense(2048, activation='relu')
Dropout_2 = tf.keras.layers.Dropout(0.5)
dense_layer_3 = tf.keras.layers.Dense(1024, activation='relu')
Dropout_3 = tf.keras.layers.Dropout(0.3)
dense_layer_4 = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')


model = tf.keras.models.Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      Dropout_1,
      dense_layer_2,
      Dropout_2,
      dense_layer_3,
      Dropout_3,
      dense_layer_4,
      prediction_layer
  ])

In [ ]:
model.load_weights('../input/eff-b0-model/eff_b0_model_224.h5')

In [ ]:
# model = keras.models.load_model('../input/eff-b2-model/eff_b2_model')

In [ ]:
test_csv = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
id_code = test_csv['id_code']
test_prediction = np.empty(len(id_code), dtype='float32')
for i in range(len(id_code)):
    img = cv2.imread('../input/aptos2019-blindness-detection/test_images/{}.png'.format(id_code[i]))
    
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = crop_image_from_gray(img).astype('uint8')
#     img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    
    img = load_ben_color(img)
    X = np.array([img])
    pred = model.predict(X)
    test_prediction[i] = np.argmax(pred).astype('int64')
test_prediction = test_prediction.astype('int64')

In [ ]:
# prediction = predict(test_prediction).astype('uint8')
test_csv['diagnosis'] = test_prediction
test_csv.to_csv("submission.csv", index=False)
unique, counts = np.unique(test_prediction, return_counts=True)
tmp = dict(zip(unique, counts))
print(tmp)
print('Done!')